<a href="https://colab.research.google.com/github/7ayden/7ayden.github.io/blob/master/hkcovid_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install streamlit
#!pip install pyngrok
#from google.colab import files
#pip freeze > requirements.txt
#files.download('requirements.txt')

In [2]:
import requests
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
from itertools import chain
import seaborn as sns
import streamlit as st
from pyngrok import ngrok

sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#Import open HK data online
r = requests.get('http://www.chp.gov.hk/files/misc/building_list_eng.csv', allow_redirects=True)
if r.ok:
    data = r.content.decode('utf8')
    bd = pd.read_csv(io.StringIO(data))
    
r = requests.get('http://www.chp.gov.hk/files/misc/building_list_chi.csv', allow_redirects=True)
if r.ok:
    data = r.content.decode('utf8')
    cbd = pd.read_csv(io.StringIO(data))


r = requests.get('http://www.chp.gov.hk/files/misc/enhanced_sur_covid_19_eng.csv', allow_redirects=True)
if r.ok:
  data = r.content.decode('utf8')
  df = pd.read_csv(io.StringIO(data))
df = df.drop(columns=['Name of hospital admitted'])
df['Case no.']= df['Case no.'].astype(int)

r = requests.get('http://www.chp.gov.hk/files/misc/large_clusters_eng.csv', allow_redirects=True)
if r.ok:
  data = r.content.decode('utf8')
  cr = pd.read_csv(io.StringIO(data))

bd = bd.reindex()
cbd = cbd.reindex()

df2= pd.merge(bd, cbd, left_index=True, right_index=True)
#### work on the splitting 
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = df2['Related probable/confirmed cases'].str.split(',').map(len)

df3 = pd.DataFrame({'Case no.': chainer(df2['Related probable/confirmed cases']),
                    'Building_name' :np.repeat(df2['Building name'], lens),
                    'District': np.repeat(df2['District'], lens),
                     'ChiBuilding_name ' :np.repeat(df2['大廈名單'], lens),
                    'Chi_District': np.repeat(df2['地區'], lens),
                    'Lastdate_residence' :np.repeat(df2['Last date of residence of the case(s)'], lens),
                    })

lens = cr['Involved case number'].str.split(',').map(len)

tcr = pd.DataFrame({'Cluster': np.repeat(cr['Cluster'], lens),
                    'Case no.': chainer(cr['Involved case number']),
                    'Number of cases' :np.repeat(cr['Number of cases'], lens),
                    })
tcr['Case no.'] = tcr['Case no.'].astype(str).astype(int)
tcr= tcr.set_index('Case no.')

#df3['Case no.']= df3['Case no.'].astype(int)
df3['Case no.'] = df3['Case no.'].astype(int)
#df3= df3.set_index('Case no.')
df4= pd.merge(df, df3, left_on='Case no.', right_on='Case no.', how='left')
df = pd.merge(df4, tcr, left_on='Case no.', right_on='Case no.', how='left')
df


,Case no.,Report date,Date of onset,Gender,Age,Hospitalised/Discharged/Deceased,HK/Non-HK resident,Case classification*,Confirmed/probable,Building_name,District,ChiBuilding_name,Chi_District,Lastdate_residence,Cluster,Number of cases
0,1,23/01/2020,21/01/2020,M,39,Discharged,Non-HK resident,Imported case,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,23/01/2020,18/01/2020,M,56,Discharged,HK resident,Imported case,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,24/01/2020,20/01/2020,F,62,Discharged,Non-HK resident,Imported case,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,24/01/2020,23/01/2020,F,62,Discharged,Non-HK resident,Imported case,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,24/01/2020,23/01/2020,M,63,Discharged,Non-HK resident,Imported case,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5147,5053,24/09/2020,Asymptomatic,F,40,To be provided,HK resident,Imported case,Confirmed,Dorsett Wanchai,Wan Chai,灣仔帝盛酒店,灣仔,NaN,NaN,NaN
5148,5054,24/09/2020,Asymptomatic,F,40,To be provided,HK resident,Imported case,Confirmed,Silka Seaview Hotel,Yau Tsim Mong,香港海景絲麗酒店,油尖旺,NaN,NaN,NaN
5149,5055,24/09/2020,21/09/2020,M,52,To be provided,HK resident,Epidemiologically linked with local case,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5150,5056,24/09/2020,20/09/2020,F,44,To be provided,HK resident,Epidemiologically linked with local case,Confirmed,"Block 7, Kenswood Court, Kingswood Villas",Yuen Long,嘉湖山莊景湖居7座,元朗,NaN,NaN,NaN


In [4]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
import altair as alt
import seaborn as sns
import altair as alt
import plotly.figure_factory as ff
import plotly.offline as py
sns.set()


st.title("Covid19 Hong Kong Open Data")

#st.header('by Jayden Yuen')



st.markdown("## " + 'TotalCases/Deaths/Recoveries Trend')	
st.markdown("#### " +"What Trends would you like to see?")

selected_metrics = st.selectbox(
    label="Choose...", options=['TotalCases','Deaths','Recoveries']
)

DATA_URL = ('https://raw.githubusercontent.com/7ayden/covid19/master/cdf2020-09-24.csv')
@st.cache(allow_output_mutation=True)
def get_data():
    data = pd.read_csv(DATA_URL)
    data['Date'] = pd.to_datetime(data['Report date']).dt.strftime('%d-%m-%Y')
    return data

#load data from url
df = get_data()
#df = df.drop(['Unnamed: 0'], axis=1, errors='ignore')
#df.set_index("Case no.", inplace = True)
District = st.sidebar.multiselect('Show which districts did you travel', df['District'].unique())

#date = st.sidebar('Show Which date were you there?', df['Report date'].unique())

# Filter dataframe
#new_df = df[(df['District'].isin(District)) & (df['Report date'].isin(date))]

new_df = df[(df['District'].isin(District))]

#new_df.sort_values(by=['Report Date'], inplace=True, ascending=False)


# create figure using plotly express
fig = px.scatter(new_df, x ='Report date',y='Age',color='Gender')

#if st.checkbox('Plot by district and Age '):
# Plot!
st.plotly_chart(fig)

#if st.checkbox('Show Historical average data'):
  # write dataframe to screen
st.write(new_df)



#clubs = st.multiselect('Show Player for clubs?', df['Club'].unique())nationalities = st.multiselect('Show Player from Nationalities?', df['Nationality'].unique())# Filter dataframe
#new_df = df[(df['Club'].isin(clubs)) & (df['Nationality'].isin(nationalities))]# write dataframe to screen

#st.write(new_df)
#df.loc[:, ~df.columns.str.contains('^Unnamed: 0')]
st.markdown("## " + 'Historical data stats')
if st.checkbox('Show Historical average data stats'):
  st.table(df[['District','Age' ]].groupby('District').agg(['mean','min','max', 'count']).sort_values(by=[('Age','mean')]))

st.markdown("## " + 'last 14 days cases')
	
import datetime as dt
df['Report date']= pd.to_datetime(df['Report date'], dayfirst = True).dt.date
range_max = df['Report date'].max()
range_min = range_max - dt.timedelta(days=14)

# take slice with final week of data
sliced_df = df[(df['Report date'] >= range_min) & 
               (df['Report date'] <= range_max)]
if st.checkbox('Show last 14 days cases data by district'):
  st.write(pd.DataFrame(sliced_df.groupby(['Report date'], sort=False)['District'].counts()))

st.markdown("## " + ' Show by district')

if st.checkbox('Show last 14 days cases data'):
  st.dataframe(sliced_df)


st.markdown("## " + ' FacetGrid')

a = sns.FacetGrid(df, hue = 'Hospitalised/Discharged/Deceased', aspect=4, palette="Set1" )
a.map(sns.kdeplot, 'Age', shade= True )
a.set(xlim=(0 , df['Age'].max()))
a.add_legend()


Hospitalised = df[df['Hospitalised/Discharged/Deceased']== 'Hospitalised']

#fig = px.bar(sliced_df, x="Report day", y="District", color="Age", barmode="group", facet_col="Gender"
#fig.show()




'''
st.markdown("## " + 'Normal Curve')
chart = px.histogram(sliced_df, x="District", y="Report date", color="Gender",
                   marginal="box", # or violin, rug
                   hover_data=sliced_df.Age)
chart
'''
st.markdown("## " + 'last 14 days cases by age and gender')



st.markdown("## " + 'last 14 days cases by age and gender')

chart2 = px.scatter(data_frame=sliced_df,
           x="Report date",
           y="Age",
           color="Gender",
           title="Last 14 days cases by age and gender")
chart2

st.markdown("## " + 'last 14 days total cases by gender')

#chart3 = px.histogram(data_frame=sliced_df,x="Report date",color="Gender", title="Last 14 days total cases count by gender")
'''
women_bins = df['Gender']== 'F'
men_bins = df['Gender']== 'M'

y = list(range(0, 100, 10))

layout = go.Layout(yaxis=go.layout.YAxis(title='Age'),
                   xaxis=go.layout.XAxis(
                       range=[0, 1200],
                       title='Number'),
                   barmode='overlay',
                   bargap=0.1)

data = [go.Bar(y=y,
               x=men_bins,
               orientation='h',
               name='Men',
               hoverinfo='x',
               marker=dict(color='powderblue')
               ),
        go.Bar(y=y,
               x=women_bins,
               orientation='h',
               name='Women',
               text=-1 * women_bins,
               hoverinfo='text',
               marker=dict(color='seagreen')
               )]
fig = go.plot(dict(data=data))
fig
#group_labels= ['Gender']
#fig = ff.create_displot(sliced_df, group_labels)
#fig
'''
st.markdown("## " + ' plotbox last 14 days total cases by gender')

chart4 = px.box(data_frame=sliced_df,
           x="Report date",
           y="Age",
           color="Gender",
           title="plotbox on the last 14 days cases")
chart4
st.markdown("## " + ' scatterplot')

chart5 = px.scatter(data_frame=df, x="Age", y=categorical("Case classification*"), facet_col="Hospitalised/Discharged/Deceased", marginal_y="violin", marginal_x="box", trendline="ols", template="simple_white")
chart5


st.markdown("## " + ' age distribution')
##trying to use count plot on age distribtuion 

fig = sns.FacetGrid(df, hue = 'Hospitalised/Discharged/Deceased', aspect=4, palette="Set1" ).map(sns.kdeplot, 'Age', shade= True ).set(xlim=(0 , df['Age'].max())).add_legend()
st.pyplot(fig)
#bar_chart(df['Gender'])



#.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))



## sidebar 
st.sidebar.title("District Data")
add_selectbox = st.sidebar.selectbox(
    "How would you like to be contacted?",
    ("Email", "Home phone", "Mobile phone")
)


# Create traces

Hospitalised = df[df["Hospitalised/Discharged/Deceased"] == "Hospitalised"]
df2 = df[df["Defect severity"] == "High"]

fig = go.Figure()
if selected_metrics == 'Hospitalised':
  fig.add_trace(go.Bar(x=Hospitalised['Report date'],y=Hospitalised['Hospitalised/Discharged/Deceased']))
if selected_metrics == 'Deaths':
	fig.add_trace(go.Bar(x=df['Report date'],y=df[df['Hospitalised/Discharged/Deceased']=='Deceased'],
	                    mode='markers', name='Deaths'))
if selected_metrics == 'Recoveries':
	fig.add_trace(go.Bar(x=df['Report date'], y=df[df['Hospitalised/Discharged/Deceased']=='Discharged'],
	                    mode='relative',
	                    name='Recoveries'))
st.plotly_chart(fig, use_container_width=True)


## By scatter plit 

go.Scatter(alt.Chart(df).mark_bar().encode(alt.X('Report data', bin=True),
	                                                          	y = 'count()'))



min_year = int(df['Age'].min())
max_year = int(df['Age'].max())

District = df['District'].unique()

'## By district'
District = st.selectbox( 'df.District',df[df['District'] == District])



filter_data = df[df['Report date'] >='2020-08-31'].set_index("Date") 
st.markdown(str(','.join(country_name_input)) + " daily Death cases from 1st April 2020")
# bar chart 
st.bar_chart(filter_data[['Deaths']])


#histogram
hist_values = np.histogram(
df['Report date'].dt.weekday_name, bins=7, range=(0,7))[0]
st.bar_chart(hist_values)

'## By month'
month = st.selectbox('Month', range(1, 13))
df[df['Report data'] == month]



#data_load_state = st.text('Loading data...')
#data_load_state.text("Done! (using st.cache)")

#st.dataframe(data2)
#st.dataframe(data3)
#st.dataframe(data4)

#cols = ["Case no.", "Report date", "Gender", "Age", "Hospitalised/Discharged/Deceased"]
#st_ms = st.multiselect("Columns", df.columns.tolist(), default=cols)

#st.table(df.groupby("Gender").Age.mean().reset_index()\
#.round(2).sort_values("Age", ascending=False)\
#.assign(avg_age=lambda x: x.pop("Age").apply(lambda y: "%.2f" % y)))

#values = st.sidebar.slider("Age range", df.Age.min()), 100, (0, 100))
#f = px.histogram(df.query(f”age.between{values}”), x=”Age”, nbins=5, title=”Age distribution”)
#f.update_xaxes(title=”Age”)
#f.update_yaxes(title=”No. of listings”)
#st.plotly_chart(f)


Writing app.py


In [5]:
!streamlit run app.py &>/dev/null&

In [6]:
#DELETE/api/tunnels/:8501

In [7]:
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

2020-09-25 05:30:11.389 INFO    pyngrok.process: ngrok process starting: 236
2020-09-25 05:30:11.414 INFO    pyngrok.process: t=2020-09-25T05:30:11+0000 lvl=info msg="no configuration paths supplied"

2020-09-25 05:30:11.418 INFO    pyngrok.process: t=2020-09-25T05:30:11+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

2020-09-25 05:30:11.424 INFO    pyngrok.process: t=2020-09-25T05:30:11+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

2020-09-25 05:30:11.429 INFO    pyngrok.process: t=2020-09-25T05:30:11+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040

2020-09-25 05:30:11.615 INFO    pyngrok.process: t=2020-09-25T05:30:11+0000 lvl=info msg="tunnel session started" obj=tunnels.session

2020-09-25 05:30:11.617 INFO    pyngrok.process: t=2020-09-25T05:30:11+0000 lvl=info msg="client session established" obj=csess id=f4bf8f0fa3a8

2020-09-25 05:30:11.629 INFO    pyngrok.process: ngrok process has s

'http://bb9dc59f72ca.ngrok.io'

Writing app.py


In [ ]:
# Exploratory Analysis Class
class ExploratoryAnalysis:

    def __init__(self, dataframe):
        self.df = dataframe
        self.columns = dataframe.columns
        self.numerical_columns = [name for name in self.columns if (self.df[name].dtype == 'int64') | (self.df[name].dtype == 'float64')]
    
    def info(self):
        buffer = io.StringIO()
        self.df.info(buf=buffer)
        return buffer.getvalue()

    def info2(self, column_target):
        df = self.df[column_target].value_counts().to_frame().reset_index()
        df.sort_values(by='index', inplace=True, ignore_index=True)
        df.rename(columns={'index':column_target, '{}'.format(column_target):"Values Frequency"}, inplace=True)
        return df

    def CountPlot(self, column_target, hue=None):
        sns.set(style="darkgrid")
        return sns.countplot(x=column_target, data=self.df, hue=hue, palette='pastel')

    def HeatMapCorr(self):
        sns.set(style="darkgrid")
        sns.set(font_scale=0.6)
        corr = self.df.corr()
        return sns.heatmap(corr, annot=True, annot_kws={"size": 7}, linewidths=.5)
   
    def DistPlot(self, column_target):
        sns.set(style="darkgrid")
        return sns.distplot(self.df[column_target], color='c')

    def PairPlot(self, hue=None):
        sns.set(style="darkgrid")
        return sns.pairplot(self.df, hue=hue,palette="coolwarm")

    def BoxPlot(self, column_x=None, column_y=None, hue=None):
        sns.set(style="darkgrid")
        return sns.boxplot(x=column_x,y=column_y,hue=hue,data=self.df, palette="Set3")



# Add a slider to the sidebar:
EA = ExploratoryAnalysis(df)
st.sidebar.title("District Data")
st.title('Dataframe basic informations')
st.sidebar.subheader('Basica exploratory analysis options')
  if st.sidebar.checkbox('Basic informations'):
    if st.sidebar.checkbox('Head'):
      st.subheader('Dataframe head:')
      st.write(df.head())

    if st.sidebar.checkbox('Describe'):
      st.subheader('Dataframe description:')
      st.write(df.describe())

    if st.sidebar.checkbox('Info'):
      st.subheader('Dataframe informations:')
      st.text(EA.info())

    if st.sidebar.checkbox('Isnull'):
      st.subheader('Null occurrences')
      st.write(df.isnull().sum())

    if st.sidebar.checkbox('Unique values and frequency'):
      col = st.sidebar.selectbox('Choose a column for see unique values',EA.columns)
      st.subheader('Unique values and frequency')
      st.write(EA.info2(col))


  st.title('Dataframe plots')
        
  st.sidebar.subheader('Data visualization options')
  if st.sidebar.checkbox('Graphics'):

    if st.sidebar.checkbox('Count Plot'):
      st.subheader('Count Plot')
      column_count_plot = st.sidebar.selectbox("Choose a column to plot count",EA.columns)
      hue_opt = st.sidebar.selectbox("Optional categorical variables (countplot hue)",EA.columns.insert(0,None))
      if st.checkbox('Plot Countplot'):
          fig = EA.CountPlot(column_count_plot, hue_opt)
          st.pyplot()

      if st.sidebar.checkbox('Distribution Plot'):
          st.subheader('Distribution Plot')
          column_dist_plot = st.sidebar.selectbox("Choose a column to plot distribution (only numerical)",EA.numerical_columns)
          if st.checkbox('Plot Distplot'):
              fig = EA.DistPlot(column_dist_plot)
              st.pyplot()
            
          if st.sidebar.checkbox('Heatmap Correlation'):
              st.subheader('Heatmap Correlation Plot')
              fig = EA.HeatMapCorr()
              st.pyplot()

          if st.sidebar.checkbox('Boxplot'):
              st.subheader('Boxplot')
              column_box_plot_X = st.sidebar.selectbox("X (Choose a column):",EA.columns.insert(0,None))
              column_box_plot_Y = st.sidebar.selectbox("Y (Choose a column - only numerical):",EA.numerical_columns)
              hue_box_opt = st.sidebar.selectbox("Optional categorical variables (boxplot hue)",EA.columns.insert(0,None))
              if st.checkbox('Plot Boxplot'):
                  fig = EA.BoxPlot(column_box_plot_X, column_box_plot_Y, hue_box_opt)
                  st.pyplot()

          if st.sidebar.checkbox('Pairplot'):
              st.subheader('Pairplot')
              hue_pp_opt = st.sidebar.selectbox("Optional categorical variables (pairplot hue)",EA.columns.insert(0,None))
              st.info("This action may take a while.")
              if st.checkbox('Plot Pairplot'):
                  fig = EA.PairPlot(hue_pp_opt)
                  st.pyplot()
    
    
    #except Exception as e:
        #st.write(e)
    except:
        st.error('Upload a csv file to get started.')

  #https://github.com/rafaelloni/Exploratory_Analysis_Tool_app/blob/master/Code/EAT.py

IndentationError: ignored

In [ ]:
'''
if st.checkbox('Show raw data'):
    st.subheader('Raw data')
    st.write(data)

st.subheader('Number of pickups by hour')
hist_values = np.histogram(data[DATE_COLUMN].dt.hour, bins=24, range=(0,24))[0]
st.bar_chart(hist_values)

# Some number in the range 0-23
hour_to_filter = st.slider('hour', 0, 23, 17)
filtered_data = data[data[DATE_COLUMN].dt.hour == hour_to_filter]

st.subheader('Map of all pickups at %s:00' % hour_to_filter)
st.map(filtered_data)
if __name__ == '__main__':
	main()
  '''

"\nif st.checkbox('Show raw data'):\n    st.subheader('Raw data')\n    st.write(data)\n\nst.subheader('Number of pickups by hour')\nhist_values = np.histogram(data[DATE_COLUMN].dt.hour, bins=24, range=(0,24))[0]\nst.bar_chart(hist_values)\n\n# Some number in the range 0-23\nhour_to_filter = st.slider('hour', 0, 23, 17)\nfiltered_data = data[data[DATE_COLUMN].dt.hour == hour_to_filter]\n\nst.subheader('Map of all pickups at %s:00' % hour_to_filter)\nst.map(filtered_data)\nif __name__ == '__main__':\n\tmain()\n  "

In [ ]:
DATA_URL = 'https://github.com/7ayden/covid19/blob/master/cdf.csv'


def load_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows)
    lowercase = lambda x: str(x).lower()
    data.rename(lowercase, axis='columns', inplace=True)
    data[DATE_COLUMN] = pd.to_datetime(data[DATE_COLUMN])
    return data

    load_data()

In [ ]:
!pip install siuba
import ipywidgets as widgets
from ipywidgets import interact
from plotnine import *

from siuba import _, filter, summarize, mutate
from siuba.data import mtcars
import pandas as pd
from plotly import tools

import warnings
warnings.filterwarnings("ignore")

jobs_gender = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-03-05/jobs_gender.csv")

     |████████████████████████████████| 112kB 2.8MB/s 


In [ ]:
def plot_pay_gap_2016(jobs_gender, major_category):
    return (
        jobs_gender
        >> filter(_.year == 2016, _.total_workers >= 20000)
        >> filter(_.major_category == major_category)
        #     >> arrange(desc(wage_percent_of_male))
        >> mutate(
            percent_female=_.workers_female / _.total_workers,
            wage_percent_female=_.total_earnings_female / _.total_earnings_male,
        )
        >> ggplot(
            aes(
                "percent_female",
                "wage_percent_female",
                color="minor_category",
                size="total_workers",
                label="occupation",
            )
        )
        + geom_point()
        + scale_size_continuous(range=[1, 10], guide=False)
        + labs(
            x="% of workforce reported as female",
            y="% of median female salary / median male",
            title="Gender disparity and pay gap in 2016",
            subtitle="Only occupations with at least 20,000 workers total",
            color="Minor category",
        )
        #        scale_x_continuous(labels = percent_format()) +
        #        scale_y_continuous(labels = percent_format())
    )

# p = plot_pay_gap_2016(jobs_gender, "Management, Business, and Financial")
# tools.mpl_to_plotly(p.draw())

In [ ]:

w = widgets.Dropdown(options=jobs_gender.major_category.unique(), value=None)

w2 = widgets.Dropdown(options=[], value=None)

def update_minor_options(*args):
    print('updating')
    w2.options = jobs_gender \
        .loc[lambda d: d.major_category == w.value, 'minor_category'] \
        .unique()

w.observe(update_minor_options, 'value')

@interact(major_category=w, minor_category = w2)
def f(major_category, minor_category):
    filtered_jobs = filter(
        jobs_gender,
        _.major_category == major_category,
        _.minor_category == minor_category
    )
    
    p = plot_pay_gap_2016(filtered_jobs, major_category)
    return tools.mpl_to_plotly(p.draw())

interactive(children=(Dropdown(description='major_category', options=('Management, Business, and Financial', '…

updating
updating
updating


In [ ]:
https://github.com/machow/tidytuesday-py/blob/master/dgrtwo-translations/app-2019-03-05-women-workplace/app-voila-workforce.ipynb